In [138]:
# Import modules
import urllib3
import certifi
import re
import json
import pymongo
import pandas
import time

# Define source link
link = 'https://www.metacritic.com/browse/movies/score/metascore/year'

# Define regex
title_regex = re.compile(r"class=\"title\"><h3>(.+)</h3>")
releaseDate_regex = re.compile(r"class=\"clamp-details\">\s+<span>(.+)<\/span>")
description_regex = re.compile(r"<div class=\"summary\">\s+(.+)\s+")
metascore_regex = re.compile(r"<div class=\"clamp-score-wrap\">\s*.*\s<div class=\"metascore_w large movie (positive|mixed|negative)\">(.*)<\/div>")
photoURL_regex = re.compile(r"<a href=\".*<img\ssrc=\"(https.*)\" alt")

# Build a python request pool
http = urllib3.PoolManager(cert_reqs = 'CERT_NONE')

# Fix SSL cert error 
urllib3.disable_warnings()

# Intiate a web request
res = http.request('GET', link, headers={'User-Agent': 'Mozilla/5.0'})

# Convert results from raw bytes to test
datastring = str(res.data, "utf-8")

# Check for success and how much data retrieved from the site
print(f'Status: {res.status}')
print(f'Fetched {len(datastring)} characters from {link}.')

Status: 200
Fetched 506429 characters from https://www.metacritic.com/browse/movies/score/metascore/year.


In [139]:
# Loading the json file that containins my secret MongoDB connection string
with open (r'C:\Users\hsely\OneDrive\Documents\GitHub\DA320\Midterm\credentials.json') as u:
    data = json.load(u)

secret_key = data['mongodb']

# We can safely print the length of the secret key. That won't leak any sensitive information.
print(f"My secret key is {len(secret_key)} characters in length.")

My secret key is 67 characters in length.


In [140]:
# Code by Vincent Hong
# Connecting to the database using known good certificates
client = pymongo.MongoClient(secret_key, tlsCAFile=certifi.where())

# Fetching my database titled "DA320"
da320_database = client.DA320
metacritic_data = da320_database.Metacritic

# Accessing my collections from my "DA320" database
allCollections = da320_database.list_collection_names()

# Print statements that display the current version as well as the collections present in DA320
print(f"Using MongoDB version {client.server_info()['version']}.")
print(f"This database has the collections {allCollections}")

Using MongoDB version 5.0.13.
This database has the collections ['Metacritic', 'IMDB', 'IMDB_Pipeline_View']


In [141]:
# Retrieve a list of movies from a particular year and page of Metacritic
def metacritic_scraper(year: int, page: int) -> pandas.DataFrame:
    
    # Fetch webpage
    url = "https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected={year}&sort=desc&view=detailed&page={page}"
    response = http.request('GET', url, headers={'User-Agent': 'Mozilla/5.0'})
    datastring = str(response.data, "utf-8")

    # Execute all regex
    titles = title_regex.findall(datastring)
    dates = releaseDate_regex.findall(datastring)
    descriptions = description_regex.findall(datastring)
    scores = metascore_regex.findall(datastring)
    images = photoURL_regex.findall(datastring)

    # Return a unified collection
    dataset = {"title": titles, "date": dates, "description": descriptions, "score": scores, "image": images}
    return pandas.DataFrame(dataset)

In [142]:
# Write a CSV file with this data
for year in range(2000, 2023):
    page = 0
    print(f"Collecting data for {year} page {page}...")

    #Retry a page multiple times if necessary
    while True:
        data = metacritic_scraper(year, page)

        # Stop at page with zero rows
        if len(data) == 0:
            break

    #Convert the dataframe into a list of movies to insert into Mongo DB
    movies_to_insert = []
    for row in data.itertuples():
        movie = {
            "title": row.title,
            "release_date": row.date,
            "description": row.description, 
            "metascore": row.score, 
            "image_url": row.image,
        }
        movies_to_insert.append(movie)
    
    # Insert records into MongoDB
    print(f"Inserting {len(movies_to_insert)} movies for the year {year} page {page}")
    metacritic_data.insert_many(movies_to_insert)
    page = page + 1


Inserting 0 movies for the year 2000 page 0


TypeError: documents must be a non-empty list

In [ ]:
# Define title to search movie Title
title = r"class=\"title\"><h3>(.+)</h3>"

# Find matches within text 
matches = re.findall(title, datastring)
print(f"Found {len(matches)} matches.  The first match is '{matches[0]}'.")

In [ ]:
# Define regular_expression to search movie Release date
releaseDate = r"class=\"clamp-details\">\s+<span>(.+)<\/span>"

# Find matches within text 
matches = re.findall(releaseDate, datastring)
print(f"Found {len(matches)} matches.  The first match is '{matches[0]}'.")

In [ ]:
# Define description to search movie Description
description = r"<div class=\"summary\">\s+(.+)\s+"

# Find matches within text 
matches = re.findall(description, datastring)
print(f"Found {len(matches)} matches.  The first match is '{matches[0]}'.")

In [ ]:
# Define metascore to search movie Metascore
metascore = r"<div class=\"clamp-score-wrap\">\s*.*\s<div class=\"metascore_w large movie (positive|mixed|negative)\">(.*)<\/div>"

# Find matches within text 
matches = re.findall(metascore, datastring)
print(f"Found {len(matches)} matches.  The first match is '{matches[0]}'.")

In [ ]:
# Define photoURL to search movie Photo URL
# Code by Vincent Hong
photoURL = r"<a href=\".*<img\ssrc=\"(https.*)\" alt"

# Find matches within text 
matches = re.findall(photoURL, datastring)
print(f"Found {len(matches)} matches.  The first match is '{matches[0]}'.")